In [ ]:
import codebase.web_scrape_functions as wsf
import codebase.analysis_functions as af
import codebase.match_data as match_data
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import utils
from utils import logger
import os
import logging
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import codebase.graphing_functions as gf
from codebase.settings import CAREERS
from codebase.settings import LABEL_DATA

if utils.check_if_ipython():
    logger.disabled = True

%load_ext autoreload
%autoreload 2
logger.handlers[1].setLevel(logging.INFO)
# logger.disabled = True
pd.get_option("display.max_columns")

First step is to get Kohli's career averges for all shots, then his averages for just specific shots and then we can find his most productive shot. The problem with searching things based off of specific shots strings only is that there is a veriety in the way that shots are described in the commentary. We would need a machine learning algorithm to work and recognize what these shots are. But first we need to create a training set which is a pain in the ass.

In [ ]:
KOHLI_ID = '253802'
ROOT_PLAYER_ID = '303669'
WILLIAMSON_PLAYER_ID = '277906'
SPD_SMITH_ID = '267192'

In [ ]:
kohli_matches = wsf.get_player_match_list(KOHLI_ID)

In [ ]:
kohli_innings = af.get_cricket_totals(KOHLI_ID, kohli_matches, _type='bat', by_innings=True, is_object_id=True, from_scorecards=False)

In [ ]:
kohli_comms = af.get_player_contributions(KOHLI_ID, kohli_matches, _type='bat', by_innings=True, is_object_id=True)

In [ ]:
def search_shots_in_comms(contributions, search_keywords, exlude_words=[], primary_keywords=[]):
    shots = []
    for i, comms in enumerate(contributions):
        innings = comms.commentTextItems.to_list()
        search = af.search_for_keywords(innings, keywords=search_keywords, exclude_words=exlude_words, primary_keywords=primary_keywords, return_matching=True, return_indices=True)
        shots.append(comms.iloc[search[2]])
    
    return shots

In [ ]:
## Cover drives 
cover_drive_search_kw = [
    'drive',
    'cover',
    'full and wide',
    'outside edge',
    'reach',
    'slip',
    'edge',
    'driving'
]
cover_drive_exlude_words = [
    'run out',
    'pull', 'flick',
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge',
    'lets one go', 'easy leave', 'leaves the ball',
    'leg side', 'leading edge', 'leg ', 'cut', 'leg.',
    'on the pads', 'left alone'
]
cover_drives_played = search_shots_in_comms(kohli_comms, cover_drive_search_kw, cover_drive_exlude_words, primary_keywords=['cover drive', 'cover-drive'])

In [ ]:
cover_drive_averages, cover_drive_totals, cover_drive_stats = af.analyse_batting(cover_drives_played)

In [ ]:
cover_drive_averages

In [ ]:
cover_drive_totals

In [ ]:
#flick
flick_primary_kw = [
    'flick'
]
flick_search_kw = [
    'clipped',
    'clips',
    'pads',
    'on the pads',
    'leg side',
    'off the pads',
    'off his pads',
    
    'square leg',
    'whip',
    'on the legs'
]
flick_exlude_words = [
    'run out', 'bumper', 'sweep',
    'pull', 'drive', 'cover-drive'
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge',
    'lets one go', 'easy leave', 'leaves the ball',
    'off side', 'cut', 'left alone'
]
flick_played = search_shots_in_comms(kohli_comms, flick_search_kw, flick_exlude_words, primary_keywords=flick_primary_kw)

In [ ]:
flicks = [x for comms in flick_played for x in comms.commentTextItems.to_list()]

In [ ]:
len(flicks)

In [ ]:
all_comms = [x for comms in kohli_comms for x in comms.commentTextItems.to_list()]

In [ ]:
with open('all_comms.txt', 'w') as file:
    for line in all_comms:
        file.writelines(line)
        file.writelines('\n')

In [ ]:
flick_primary_kw = [
    'flick'
]
flick_search_kw = [
    'tuck',
    'clipped',
    'clip',
    ('pads', 0.3),
    ('pad ', 0.3),
    'on the pads',
    #('leg side', 0.25),
    ('leg', 0.3),
    ('square', 0.19),
    'glance'
    'off the pads',
    'off his pads',
    ('nudge', 0.3),
    ('pick', 0.3),
    ('straight', 0.3),
    #('square leg', 0.25),
    ('whip', 0.4),
    'on the legs',
    ('work', 0.25),
    ('midwicket', 0.3),
    ('on side', 0.25),
    ('across', 0.3)
]
flick_exlude_words = [
    'run out', 'bumper', 'sweep', 'swept',
    'pull', 'drive', 'cover-drive', ('cover', 0.3),
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge',
    'lets one go', 'easy leave', 'leaves the ball', 'padded away',
    ('off side', 0.25), 'cut', 'left alone', ('point', 0.25), ('defend', 0.3), ('push', 0.25)
]

kohli_comms_list = [x for comms in kohli_comms for x in comms.commentTextItems.to_list()]



In [ ]:
kohli_comms_list[5]

In [ ]:
ind, scores = af.search_for_phrases(kohli_comms_list, flick_search_kw, flick_exlude_words, flick_primary_kw, threshold=0.49)

In [ ]:
ind, scores = af.search_for_phrases(["Wasted bouncer, down the leg side, easy to stay out of harm's way here"], flick_search_kw, flick_exlude_words, flick_primary_kw,)

In [ ]:
flicks_p = [kohli_comms_list[i] for i,val in enumerate(ind) if val == 1]

In [ ]:
len(flicks_p)

In [ ]:
possible_flicks = [kohli_comms_list[i] for i,val in enumerate(scores) if val == 0.25]

In [ ]:
possible_flicks

In [ ]:
flicks_p_v1 = flicks_p
possible_flicks_v1 = possible_flicks

In [ ]:
len(flicks_p)

In [ ]:
len(flicks_p_v1)

In [ ]:
list(set(flicks_p)-set(flicks_p_v1))

In [ ]:
flick_primary_kw = [
    'flick'
]
flick_search_kw = [
    'tuck',
    'clipped',
    'clip',
    ('pads', 0.3),
    ('pad ', 0.3),
    'on the pads',
    #('leg side', 0.25),
    ('leg', 0.3),
    ('square', 0.19),
    'glance'
    'off the pads',
    'off his pads',
    ('nudge', 0.3),
    ('pick', 0.3),
    ('straight', 0.3),
    #('square leg', 0.25),
    ('whip', 0.4),
    'on the legs',
    ('work', 0.25),
    ('midwicket', 0.3),
    ('on side', 0.25),
    ('across', 0.3)
]
flick_exlude_words = [
    'run out', 'bumper', 'sweep', 'swept',
    'pull', 'drive', 'cover-drive', ('cover', 0.3),
    'bouncer', 'short ball', 'stays back',
    'backfoot', 'top edge',
    'lets one go', 'easy leave', 'leaves the ball', 'padded away',
    ('off side', 0.25), 'cut', 'left alone', ('point', 0.25), ('defend', 0.3), ('push', 0.25)
]

kohli_comms_list = [x for comms in kohli_comms for x in comms.commentTextItems.to_list()]

In [ ]:
ind, scores = af.search_for_phrases(["Big lbw appeal that amuses kohli. anderson traps him on the pad but it's a big stride and he's well outside off as this shapes back in"], flick_search_kw, flick_exlude_words, flick_primary_kw, threshold=0.49)

In [ ]:
scores